In [ ]:
pip install pydub

In [ ]:
from pydub import AudioSegment
import numpy as np
from functools import reduce


class Silence:
  numChannels,frameRate,sampleWidth=2,44100,2
  audio,silenceAmplitude=None,372
  finalData,intersectedData=[],[]

  def __init__(self,audioseg,sa):
    finalData=[]
    self.audio=audioseg
    self.silenceAmplitude=sa
    self.numChannels=self.audio.channels
    self.frameRate=self.audio.frame_rate
    self.sampleWidth=self.audio.sample_width
    rawData=np.array(self.audio.get_array_of_samples())
    l=rawData.reshape(-1,self.numChannels)
    r=len(l)%self.sampleWidth
    print(self.audioMetaData())
    print(r ,'samples extra')

    for i in range(self.numChannels):
      datai=l[r:,i]
      datai=np.where(abs(datai) <=self.silenceAmplitude)[0]
      self.finalData.append(datai)
    print('Intersection in Progress')
    self.intersectedData=reduce(np.intersect1d, self.finalData)

  def getSilentIndices(self):
    return self.intersectedData

  def audioMetaData(self):
    return self.numChannels,self.frameRate,self.sampleWidth

class Interval:
    freq=44100
    def __init__(self, startIndex,freq):
        self.startIndex = startIndex
        self.intervalLength = 1
        self.endIndex = startIndex
        self.freq=freq
        self.startTime=self.getStartTime()

    def increaseLength(self):
        self.endIndex += 1
        self.intervalLength += 1

    def getStartTime(self):
        result_in_seconds=self.startIndex/self.freq
        minutes = int(result_in_seconds // 60)
        seconds = result_in_seconds % 60
        #seconds=round(seconds,1)
        return [minutes,seconds]

    def getEndTime(self):
        result_in_seconds=self.endIndex/self.freq
        minutes = int(result_in_seconds // 60)
        seconds = result_in_seconds % 60
        #seconds=round(seconds,1)
        return [minutes,seconds]

    def getIntervalLength(self):
        return self.intervalLength/self.freq


class IntervalArray:
    silent=None
    def __init__(self, silent):
        self.data = silent.getSilentIndices()
        self.silent=silent

    def getIntervalArray(self):
        #self.data = np.genfromtxt(self.file_path)
        current = Interval(self.data[0],self.silent.frameRate)
        l = []

        for i in range(len(self.data)-1):
            if (self.data[i] + 1 == self.data[i+1]):
                current.increaseLength()
            else:
                l.append(current)
                current = Interval(self.data[i+1],self.silent.frameRate)

        l.append(current)
        return l


class Filter:
  minDuration=None,372,1
  IntervalArray=[]
  def __init__(self,s):
    self.IntervalArray=IntervalArray(s).getIntervalArray()

  def filterByTime(self,t):
    filtered=[]
    for interval in self.IntervalArray:
      if(interval.getIntervalLength()>=t):
        filtered.append(interval)
    return filtered

  def isContinous(self,current,next):
    b=False
    s1=current.getEndTime()
    s2=next.getStartTime()
    if( s1[0]== s2[0] and (round(s2[1],1)==round(s1[1],1)) ):
     b=True
    return b


  def removeDuplicateIntervals(self,intervals):
    newIntervals=[]
    currentNewInterval=NewInterval(intervals[0].getStartTime(),intervals[0].getEndTime())
    for i in range(len(intervals)-1):
      current=intervals[i]
      next=intervals[i+1]
      if(self.isContinous(current,next)):
        currentNewInterval.updateEndTime(next.getEndTime())
      else:
        newIntervals.append(currentNewInterval)
        currentNewInterval=NewInterval(next.getStartTime(),next.getEndTime())
    newIntervals.append(currentNewInterval)
    return newIntervals

class NewInterval:
  startTime,endTime,intervalLength=[],[],[]

  def __init__(self,st,end):
    self.startTime=st
    self.endTime=end
    self.intervalLength=[end[0]-st[0],round(end[1]-st[1],3)]

  def updateStartTime(self,newStartTime):
    self.startTime=newStartTime

  def updateEndTime(self,newEndTime):
    self.endTime=newEndTime

  def getIntervalLength(self):
    return [self.endTime[0]-self.startTime[0],round(self.endTime[1]-self.startTime[1],3)]

  def getStartTime(self):
    return [round(self.startTime[0],1),round(self.startTime[1],1)]

  def getEndTime(self):
    return [round(self.endTime[0],1),round(self.endTime[1],1)]

class Reference:
  fp,maxAmplitude,intialMinDuration=None,372,1
  ref=[]
  def __init__(self,f,m,i):
    self.fp=f
    self.maxAmplitude=m
    self.intialMinDuration=i
    a=AudioSegment.from_file(self.fp)
    si=Silence(a,self.maxAmplitude)
    fil=Filter(si)
    il=self.intialMinDuration
    while(il>=0.0005):
      ia=fil.filterByTime(il)
      d=fil.removeDuplicateIntervals(ia)
      self.ref.append(d)
      il=il/2

  def getReference(self):
    return self.ref

In [ ]:
fpath=('/content/drive/MyDrive/Music/Old/Ilayaraja songs 1975 _ 80(MP3_320K).mp3')
ro=Reference(fpath,372,1)
reference=ro.getReference()
for r in reference:
  print('\n\n')
  for p in r:
    print('Start Time :',p.getStartTime(),'End Time :',p.getEndTime(),'Interval Length :',p.getIntervalLength())